<a href="https://colab.research.google.com/github/ailunguo/Test/blob/main/%E6%A8%A1%E5%9E%8BAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 类模型

In [ ]:
import tensorflow as tf
import keras

In [ ]:
# 1
# tf.keras.Model()
inputs = tf.keras.Input(shape=(3,))
x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
# 示例
inputs = keras.Input(shape=(None, None, 3))
processed = keras.layers.RandomCrop(width=32, height=32)(inputs)
conv = keras.layers.Conv2D(filters=2, kernel_size=3)(processed)
pooling = keras.layers.GlobalAveragePooling2D()(conv)
feature = keras.layers.Dense(10)(pooling)

full_model = keras.Model(inputs, feature)
backbone = keras.Model(processed, conv)
activations = keras.Model(conv, feature)

In [ ]:
# 2，模型子类化
class MyModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense2(5, activation=tf.nn.softmax)
    # x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
    # outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x)

  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(x)
model = MyModel()

In [ ]:
# 可以在call()中包含一个training布尔参数来指定不同的行为
class MyModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)
    self.dropout = tf.keras.layers.Dropout(0.5)

  def call(self, inputs, training=False):
    x = self.dense1(inputs)
    if training:
      x = self.dropout(x, training=training)
    return self.dense2(x)

model = MyModel()

### summary方法

In [ ]:
Model.summary(
    line_length=None, # 打印行的总长度
    positions=None,  # 每行中日志元素的相对和绝对位置
    print_fn=None,   #
    expand_nested=False,
    show_trainable=False, # 是否显示图层是否可训练
    layer_range=None,
)

In [ ]:
# get_layer方法
Model.get_layer(name=None, index=None)
# name: 字符串，图层的名称
# index: 整数，图层的索引

# Sequential类

In [ ]:
tf.keras.Sequential(layers=None, name=None)

In [ ]:
# 示例
# 由于没有指定激活函数，默认使用线性激活函数
model = tf.keras.Sequential()
# 第一个层要说明输入层的维度
model.add(tf.keras.layers.Dense(8, input_shape=(16,)))
model.add(tf.keras.layers.Dense(4))

# 下面的与上面的类似
mdoel = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(16,)))
model.add(tf.keras.layers.Dense(8))

# 也可以忽略输入
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8))
model.add(tf.keras.layers.Dense(4))
# 但是模型的权重不会被创建

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8, input_shape=(16,)))
model.add(tf.keras.layers.Dense(4))
len(model.weights) == 4 # 两个层，分别两两个权重和两个偏差

# 如果不添加输入层，那权重需要手动创建，并指定输入层
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8))
model.add(tf.keras.layers.Dense(4))
model.build((None, 16))
len(model.weights) == 4

# 如果没有手动创建，那么在fit,eval或给模型输入数据时会自动创建
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='sgd', loss='mse')
model.fit(x, y, batch_size=32, epochs=10) # 这个操作会自动创建权重

In [ ]:
Sequential.pop() # 删除模型的最后一层

# 模型训练API

In [ ]:
# compile方法
Model.compile(
    optimizer='rmsprop',
    loss=None,
    metrics=None,
    loss_weights=None,
    weighted_metrics=None,
    run_eagerly=None,
    steps_per_execution=None,
    jit_compile=None,
    pss_evaluation_shards=0,
    **kwargs
)

In [ ]:
# 示例
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
       loss=tf.keras.losses.BinaryCrossentropy(),
       metrics=[tf.keras.metrics.BinaryAccuracy(),
            tf.keras.metrics.FalseNegatives()])

optimizer: 字符串，优化器名称或优化器示例
loss: 损失函数
metrics: 模型在训练和测试期间要评估的指标列表
loss_weights: 列表或字典，以对不同模型输出的损失贡献加权。
weighted_metrics: 训练和测试期间要评估和加权的指标列表
run_eagerly: bool值
steps_per_execution: 整数，每次调用期间要运行的批次数
jit_compile: 如果为true使用XLA编译模型训练步骤，XLA是一个用于机器学习的优化编译器
pss_evaluation_shards: 整数

In [ ]:
# fit()方法
Model.fit(
    x=None,
    y=None,
    batch_size=None,
    epochs=1,
    verbose='auto',
    callback=None,
    validation_split=0.0
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)

verbose: 'auto',0,1,2。0=静默，1=进度条，2=每轮一行，在大多数情况下'auto'变为1。在生产环境中，建议使用2。'auto'与ParameterServerStrategy一起时为2

valid_split: 验证集的比例

validation_data：用于评估每个时期结束时的损失和任何模型指标的数据。该模型不会根据此数据进行训练

In [ ]:
# evaluate方法
Model.evaluate(
    x=None,
    y=None,
    batch_size=None,
    verbose='auto',
    sample_weight=None,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
    return_dict=False,
    **kwargs
)

In [ ]:
# predict方法
Model.predict(
    x,
    batch_size=None,
    verbose='auto',
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)

In [ ]:
# train_on_batch方法
Model.train_on_batch(
    x,
    y=None,
    sample_weight=None,
    calss_weight=None,
    reset_metrics=True,
    return_dict=False,
)
# 对单批数据运行单个梯度更新


In [ ]:
# test_on_batch方法
Model.test_on_batch(
    x, y=None,
    sample_weight=None,
    reset_metrics=True,
    return_dict=False
)
# 在单批样品上测试模型

In [ ]:
# predict_on_batch方法
Model.predict_on_batch(x)
# 返回单批样本的预测

In [ ]:
# run_eagerly
# tf.keras.Model.run_eagerly可设置属性，指示模型是否应立即运行，
# 急切的运行代码意味着你的模型将像Python代码一样一步步运行。
# 运行速度较慢，但通过单步执行各个层调用，更容易调试它
# 默认情况下，我们将尝试将您的模型编译为静态图，以提供最佳的执行性能。

# 模型保存

In [ ]:
# 整个模型保存和加载
# save方法
Model.save(filepath, overwrite=True, save_format=None, **kwargs)
# overwrite: 我们是否应该覆盖目标位置的任何现有模型
# save_format: 'keras','tf','h5',模型文件保存的模式

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(5, input_shape=(3,)),
    tf.keras.layers.Softmax()
])
model.save('model.keras')
loaded_model = tf.keras.models.load_model('model.keras')
x = tf.random.uniform((10, 3))
assert np.allclose(model.predict(x), loaded_model.predict(x))

In [ ]:
# save_model函数
tf.keras.saving.save_model(
    model, filepath, overwrite=True, save_format=None, **kwargs
)


In [ ]:
# load_model函数
tf.keras.saving.load_model(
    filepath, custom_objects=None, compile=True, safe_mode=True, **kwargs
)
# safe_mode: 确保代码不被随意执行，默认为True

只保存权重

In [ ]:
# get_weights方法
Model.get_weights()

# set_weights方法
Model.set_weights(weights) # 为model设置weights权重

In [ ]:
import tensorflow as tf

In [ ]:
layer_a = tf.keras.layers.Dense(1,
      kernel_initializer=tf.constant_initializer(1.))
a_out = layer_a(tf.convert_to_tensor([[1., 2., 3.]]))
layer_a.get_weights() # 得到layer_a的权重

[array([[1.],
        [1.],
        [1.]], dtype=float32),
 array([0.], dtype=float32)]

In [ ]:
layer_b = tf.keras.layers.Dense(1,
      kernel_initializer=tf.constant_initializer(2.))
b_out = layer_b(tf.convert_to_tensor([[10., 20., 30.]]))
layer_b.get_weights() # 得到layer_b的权重

[array([[2.],
        [2.],
        [2.]], dtype=float32),
 array([0.], dtype=float32)]

In [ ]:
# 通过set_weights将layer_b的权重设置为layer_a的权重
# 前提是这两个模型的权重的shape是一样的
layer_b.set_weights(layer_a.get_weights())
layer_b.get_weights()

[array([[1.],
        [1.],
        [1.]], dtype=float32),
 array([0.], dtype=float32)]

In [ ]:
# save_weights方法
Model.save_weights(filepath, overwrite=True, save_format=None, options=None)

In [ ]:
# load_weights方法
Model.load_weights(filepath, skip_mismatch=False, by_name=False, options=None)
# by_name=True,如果权重以.h5形式保存，在加载权重时可以通过by_name=True。
# 这种形式下只有layers的名称相同才会被加载，这对模型微调和迁移学习非常有用，当一些层被改变之后


模型配置序列化

In [ ]:
# get_config方法
Model.get_config()

# from_config方法
Model.from_config(config, custom_objects=None)

In [ ]:
# clone_model函数
tf.keras.models.clone_model(model, input_tensors=None, clone_function=None)

# 例子
model = keras.Sequential([
    keras.Input(shape=(728,)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='softmax'),
])
# 创建一个新的模型并初始化新的权重
new_model = clone_model(model)

模型导出用于推理

In [ ]:
# ExportArchive类
tf.keras.export.ExportArchive()

# 例子
export_archive = ExportArchive()
export_archive.track(model)
export_archive.add_endpoint(
    name='serve',
    fn=model.call,
    input_signature=[tf.TensorSpec(shape=(None,3), dtype=tf.float32)],
)
export_archive.write_out('path/to/location')

serving_model = tf.saved_model.load('path/to/location')
outputs = serving_model.serve(inputs)

In [ ]:
# 下面介绍了如何导出一个模型，该模型具有一个用于推理的端点和一个用于训练模式前向传播的端点
export_archive = ExportArchive()
export_archive.track(model)
export_archive.add_endpoint(
    name='call_inference',
    fn=lambda x: model.call(x, training=False),
    input_signature=[tf.TensorSpec(shape=(None, 3), dtype=tf.float32)],
)
export_archive.add_endpoint(
    name='call_training',
    fn=lambda x: model.call(x, training=True),
    input_signature=[tf.TensorSpec(shape=(None, 3), dtype=tf.float32)],
)
export_archive.write_out('path/to/location')
